In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import os
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_auth
import plotly.graph_objects as go
import plotly.express as px

class db:
    def __init__(self):
        self.transactions = db.transaction_init()
        self.cc = pd.read_csv(r'db\country_codes.csv',index_col=0)
        self.customers = pd.read_csv(r'db\customers.csv',index_col=0)
        self.prod_info = pd.read_csv(r'db\prod_cat_info.csv')
        
    @staticmethod
    def transaction_init():
        transactions = pd.DataFrame()
        src = r'db\transactions'
        for filename in os.listdir(src):
            transactions = transactions.append(pd.read_csv(os.path.join(src,filename),index_col=0))

        def convert_dates(x):
            try:
                return dt.datetime.strptime(x,'%d-%m-%Y')
            except:
                return dt.datetime.strptime(x,'%d/%m/%Y')

        transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))

        return transactions

    def merge(self):
        df = self.transactions.join(self.prod_info.drop_duplicates(subset=['prod_cat_code'])
        .set_index('prod_cat_code')['prod_cat'],on='prod_cat_code',how='left')

        df = df.join(self.prod_info.drop_duplicates(subset=['prod_sub_cat_code'])
        .set_index('prod_sub_cat_code')['prod_subcat'],on='prod_subcat_code',how='left')

        df = df.join(self.customers.join(self.cc,on='country_code')
        .set_index('customer_Id'),on='cust_id')

        self.merged = df
        
df = db()
df.merge()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# 1
grouped = df.merged.groupby([df.merged['tran_date'].dt.day_name(),'Store_type']).sum().reset_index()    
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
grouped.tran_date = pd.Categorical(grouped.tran_date,categories=weekdays)
grouped = grouped.sort_values('tran_date')
    
data =  px.bar(grouped, x="tran_date", y="total_amt", color="Store_type",barmode="group",color_discrete_map={
                "Flagship store": "MediumSlateBlue", "e-Shop": "LightBlue", "TeleShop": "LightGray", "MBR":"MediumPurple"} ,
              labels={"tran_date": "Dzień tygodnia",  "total_amt": "Sprzedaż", "Store_type": "Kanał sprzedaży"},
title="Sprzedaż w poszczególnych dniach tygodnia z podziałem na kanały sprzedaży" )

data.update_yaxes( 
                tickprefix="$", showgrid=True
                )

data.update_layout(
                font_family="Arial",
                legend=dict(
                 title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
                    )
                    )
    
fig = go.Figure(data=data,layout = go.Layout(title='Sprzedaż w poszczególnych dniach tygodnia z podziałem na kanały sprzedaży'))           
    
# 2
df.merged['DOB'] =  pd.to_datetime(df.merged['DOB'])
df.merged['Age'] = round((df.merged['tran_date'] - df.merged['DOB'])/np.timedelta64(1,'Y'))
age_by_store = df.merged[['Store_type','Age']].groupby('Store_type').value_counts().to_frame().reset_index()

data1 = px.histogram(data_frame = age_by_store ,x = 'Age', y=0 ,color = 'Store_type' ,nbins=20,color_discrete_map={
                "Flagship store": "MediumSlateBlue", "e-Shop": "LightBlue", "TeleShop": "LightGray", "MBR":"MediumPurple"},
              labels={"Age": "Wiek klientów", "Store_type": "Kanał sprzedaży", "0":"Ilość klientów"},
title="Wiek klientów w poszczególnych kanałach sprzedaży" )

data1.update_yaxes( 
                title='Ilość klientów'
                )

data1.update_layout(
                font_family="Arial",
                legend=dict(
                 title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
                    )
                    )

fig1 = go.Figure(data=data1,layout= go.Layout(title='Wiek klientów w poszczególnych kanałach sprzedaży'))

app.layout = html.Div(children=[
            html.H1('Kanały sprzedaży',style={'text-align':'center'}),
                        html.Div([html.Div([dcc.Graph(id='first',figure=fig)],style={'width':'50%'}),
                        html.Div([dcc.Graph(id='second',figure=fig1)],style={'width':'50%'})],style={'display':'flex'})
                        ])
    

if __name__ == '__main__':
    app.run_server()